In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/RANC/software

/content/drive/MyDrive/RANC/software


In [3]:
!pip install ./tealayers/tealayer2.0/


Processing ./tealayers/tealayer2.0
  Preparing metadata (setup.py) ... done
  Created wheel for tealayer2: filename=tealayer2-2.0-py3-none-any.whl size=9010 sha256=13a09c90e1e8680a3075a6be0fc82a1b9e4fdf0736024d16c276cec033c5bf6a
  Stored in directory: /tmp/pip-ephem-wheel-cache-o6g03hba/wheels/8b/52/4c/1dcaed17af4c15154718083a8af454a895a4a4d3528c690b18
Successfully built tealayer2
  Attempting uninstall: tealayer2
    Found existing installation: tealayer2 2.0
    Uninstalling tealayer2-2.0:
      Successfully uninstalled tealayer2-2.0


In [4]:
!pip install ./rancutils/

Processing ./rancutils
  Preparing metadata (setup.py) ... done
  Created wheel for rancutils: filename=rancutils-0.1-py3-none-any.whl size=12918 sha256=7a69ac62e80140b75e8406f178095b387c53c42814ce2af94bf2fbf796c2d972
  Stored in directory: /tmp/pip-ephem-wheel-cache-hled94j1/wheels/c1/2c/b7/78cae1a5e6b1ec6c0ce2a87e8df680c25c2a9d80dc258c23f1
Successfully built rancutils
  Attempting uninstall: rancutils
    Found existing installation: rancutils 0.1
    Uninstalling rancutils-0.1:
      Successfully uninstalled rancutils-0.1


In [5]:
from tensorflow.keras.optimizers.legacy import Adam
from tealayer2 import Tea, AdditivePooling
from tensorflow.keras.layers import Flatten, Activation, Input, Lambda, concatenate
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers.legacy import Adam
from keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model
import tensorflow.compat.v1 as tf
import numpy as np
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
num_test_samples = 10000

In [9]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Greyscale images are of shape (28,28,1)
inputs = Input(shape=(28,28,1))

# Flatten the inputs so that inputs map as: flatten_input[0] -> axon[0], ..., flatten_input[255] -> axon[255]
cropped_inputs = Lambda(lambda x: x[:, 1:27, 1:27, :])(inputs)
flattened_inputs = Flatten()(cropped_inputs)

# Generate each core.
# We are taking a 16x16 square of the input image and striding it by 12. this gives us 4 cores with 0 padding encumpassing the entire image.
# Trải phẳng các ảnh input 28x28x1 thành 1 vector 784x1, quét từng đoạt 256 để đưa vào lần lượt các core, mỗi đoạn sau stride 12 phần tử so với đoạn trước
# (Trong chương 02 ở file doc là quét 4 góc bức ảnh, nó cũng tương tự như thế này)
core0 = Lambda(lambda x : x[:, :256])(flattened_inputs)
core1 = Lambda(lambda x : x[:, 140:396])(flattened_inputs)
core2 = Lambda(lambda x : x[:, 280:536])(flattened_inputs)
core3 = Lambda(lambda x : x[:, 420:])(flattened_inputs)

# Use the image distributions as corresponding inputs into our Tea Layer.
# units là số neuron được sử dụng cho tea layer này, ở đây khai báo 4 tea layer với units = 64 (tương ứng 4 core với mỗi core sử dụng 64 neuron)
core0 = Tea(units=128, name='tea_1_1')(core0)
core1 = Tea(units=128, name='tea_1_2')(core1)
core2 = Tea(units=128, name='tea_1_3')(core2)
core3 = Tea(units=128, name='tea_1_4')(core3)


# The classification is the concatenation of these 4 core's outputs.
# We'll call the classification core our 'network'
layer1 = concatenate([core0, core1, core2, core3])

network0 =  Lambda(lambda x : x[:, :256])(layer1)
network1 =  Lambda(lambda x : x[:, 256:])(layer1)


network0 = Tea(units=128, name='tea_2_1')(network0)
network1 = Tea(units=128, name='tea_2_2')(network1)
layer2 = concatenate([network0, network1])

# Gộp đầu ra của 4 core trên lại thành 1 layer khác (tổng sẽ là 256 đầu vào cho tea layer này), tuy nhiên
# do chỉ có 10 class (các số từ 0 đến 9, mỗi số 1 class) => layer này chỉ sử dụng 250 neuron (do 256 không
# chia hết cho 10), trong đó cứ 25 neuron thì sẽ vote cho 1 class, số lượng neuron vote cho class nào lớn
# nhất thì ảnh sẽ thuộc về class đó. Ví dụ trong 250 đầu ra, từ 0 đến 24 có 17 spike bắn ra, đồng thời
# không có cụm nào bắn ra được nhiều bằng hoặc hơn 17 => Ảnh thuộc về class số "0"
layer3 = Tea(units=250, name='tea_3')(layer2)

network = AdditivePooling(10)(layer3)

predictions = Activation('softmax')(network)

model = Model(inputs=inputs, outputs=predictions)

# Define a learning rate schedule
def lr_schedule(epoch):
    if epoch <= 8:
        return 0.003
    if epoch <= 15:
        return 0.002
    else:
        return 0.001
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', min_delta =0, patience=30, verbose=1, restore_best_weights=True)

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

model.fit(X_train, y_train, batch_size=128, epochs=100, verbose=1, validation_split=0.2, callbacks=[LearningRateScheduler(lr_schedule), early_stopping])

# model.fit(X_train, y_train, batch_size=128, epochs=15,verbose=1, validation_split=0.2)

score = model.evaluate(X_test, y_test, verbose=0)

print("Test Loss: ", score[0])
print("Test Accuracy: ", score[1])

Train on 48000 samples, validate on 12000 samples
Epoch 1/100
47872/48000 [============================>.] - ETA: 0s - loss: 1.2315 - acc: 0.4904

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


48000/48000 [==============================] - 12s 260us/sample - loss: 1.2295 - acc: 0.4914 - val_loss: 0.3720 - val_acc: 0.8932 - lr: 0.0030
Epoch 2/100
48000/48000 [==============================] - 8s 161us/sample - loss: 0.2720 - acc: 0.9233 - val_loss: 0.2411 - val_acc: 0.9286 - lr: 0.0030
Epoch 3/100
48000/48000 [==============================] - 6s 121us/sample - loss: 0.2111 - acc: 0.9390 - val_loss: 0.2187 - val_acc: 0.9357 - lr: 0.0030
Epoch 4/100
48000/48000 [==============================] - 5s 101us/sample - loss: 0.1815 - acc: 0.9498 - val_loss: 0.2007 - val_acc: 0.9469 - lr: 0.0030
Epoch 5/100
48000/48000 [==============================] - 7s 155us/sample - loss: 0.1603 - acc: 0.9562 - val_loss: 0.1968 - val_acc: 0.9473 - lr: 0.0030
Epoch 6/100
48000/48000 [==============================] - 5s 101us/sample - loss: 0.1467 - acc: 0.9592 - val_loss: 0.1810 - val_acc: 0.9522 - lr: 0.0030
Epoch 7/100
48000/48000 [==============================] - 5s 111us/sample - loss: 0.13

In [11]:
# Optionally, then save the generated network out for use in the simulator and/or hardware
from rancutils.teaconversion import create_cores, create_packets, Packet
from rancutils.output_bus import OutputBus
from rancutils.serialization import save as sim_save

x_test_flat = X_test.reshape((10000, 784))
x_test_cropped = X_test[:, 1:27, 1:27, :].reshape((num_test_samples, 676))
partitioned_packets = []

# Use absolute/hard reset by specifying neuron_reset_type=0
cores_sim = create_cores(model, 3, neuron_reset_type=0)
# Partition the packets into groups as they will be fed into each of the input cores
partitioned_packets.append(x_test_cropped[:num_test_samples, :256])
partitioned_packets.append(x_test_cropped[:num_test_samples, 140:396])
partitioned_packets.append(x_test_cropped[:num_test_samples, 280:536])
partitioned_packets.append(x_test_cropped[:num_test_samples, 420:])
packets_sim = create_packets(partitioned_packets)
output_bus_sim = OutputBus((0, 3), num_outputs=250)

# This file can then be used as an input json to the RANC Simulator through the "input file" argument.
sim_save("mnist_config.json", cores_sim, packets_sim, output_bus_sim, indent=2)
# Additionally, output the tensorflow predictions and correct labels for later cross validation
predict = model.predict(X_test[:num_test_samples,:])
idx=[]
for i in predict:
  idx.append(np.argmax(i))
test_predictions = to_categorical(idx)
np.save("mnist_tf_preds.txt", test_predictions)
np.save("mnist_correct_preds.txt", y_test)

#print(partitioned_packets)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Sửa file neuron trong rancutils

In [12]:
from rancutils.serialization import load
from rancutils.emulation import output_for_streaming,output_for_testbench

In [13]:
packets,cores=load("mnist_config.json")

In [14]:
output_for_streaming(cores,packets,max_xy=(4,3))

In [15]:
correct_outputs=np.load("mnist_correct_preds.txt.npy")
output_for_testbench(packets,correct_outputs)